In [1]:
import json
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as snspy

Matplotlib is building the font cache; this may take a moment.


# PS Standard Test

In [2]:
import json
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


def extract_benchmark_data():
    """
    Extract benchmark data for both proving and verification times from PS protocol benchmarks
    """
    results = []
    base_path = Path("../target/criterion/ps_equality_split")

    print(f"Analyzing benchmark data from: {base_path}")

    # Look for both prove and verify benchmarks
    prove_dirs = list(base_path.glob("prove_credentials_*"))
    verify_dirs = list(base_path.glob("verify_credentials_*"))

    print(
        f"\nFound {len(prove_dirs)} proving and {len(verify_dirs)} verification configurations")

    # Process proving benchmarks
    for bench_dir in prove_dirs:
        # Extract credential count from directory name
        creds = int(bench_dir.name.split('_')[2])

        estimates_file = bench_dir / "new" / "estimates.json"
        print(f"\nAnalyzing proving configuration: {creds}")
        print(f"Reading from: {estimates_file}")

        if estimates_file.exists():
            try:
                with open(estimates_file, 'r') as f:
                    data = json.load(f)

                # Extract timing metrics (converting nanoseconds to milliseconds)
                mean_time = data['mean']['point_estimate'] / 1_000_000
                std_dev = data['mean']['standard_error'] / 1_000_000

                # Find corresponding verification time
                verify_dir = next(d for d in verify_dirs if int(
                    d.name.split('_')[2]) == creds)
                verify_estimates = verify_dir / "new" / "estimates.json"

                with open(verify_estimates, 'r') as f:
                    verify_data = json.load(f)

                verify_mean = verify_data['mean']['point_estimate'] / 1_000_000
                verify_std = verify_data['mean']['standard_error'] / 1_000_000

                results.append({
                    'Credentials': creds,
                    'Prove Time (ms)': mean_time,
                    'Prove Std Dev': std_dev,
                    'Verify Time (ms)': verify_mean,
                    'Verify Std Dev': verify_std,
                    'Total Time (ms)': mean_time + verify_mean
                })
                print(f"✓ Successfully processed data for {creds} credentials")

            except Exception as e:
                print(f"× Error processing data: {e}")

    if results:
        # Create DataFrame and sort by credential count
        df = pd.DataFrame(results)
        df = df.sort_values('Credentials')

        # Save results
        df.to_csv('ps_credential_scaling.csv', index=False)
        df.to_json('ps_credential_scaling.json', orient='records', indent=2)

        print("\nScaling Analysis Summary:")
        print("-" * 70)
        print(df.to_string(index=False))

        return df
    else:
        print("\nNo data was collected!")
        return None


def analyze_scaling(df):
    """Analyze and visualize scaling behavior for PS protocol proving and verification"""
    plt.figure(figsize=(12, 8))

    # Plot proving time with error bars
    plt.scatter(df['Credentials'], df['Prove Time (ms)'],
                color='blue', label='Proving Time', s=100)
    plt.errorbar(df['Credentials'], df['Prove Time (ms)'],
                 yerr=df['Prove Std Dev'], fmt='none',
                 capsize=5, color='blue', alpha=0.5)

    # Plot verification time with error bars
    plt.scatter(df['Credentials'], df['Verify Time (ms)'],
                color='red', label='Verification Time', s=100)
    plt.errorbar(df['Credentials'], df['Verify Time (ms)'],
                 yerr=df['Verify Std Dev'], fmt='none',
                 capsize=5, color='red', alpha=0.5)

    # Plot total time
    plt.scatter(df['Credentials'], df['Total Time (ms)'],
                color='green', label='Total Time', s=100)

    # Compute and plot ideal linear scaling for both operations
    base_creds = df['Credentials'].iloc[0]
    x_line = np.linspace(0, df['Credentials'].max() * 1.1, 100)

    # Proving linear scaling
    base_prove = df['Prove Time (ms)'].iloc[0]
    y_prove = (base_prove/base_creds) * x_line
    plt.plot(x_line, y_prove, '--', color='blue',
             label='Ideal Linear (Prove)', alpha=0.5)

    # Verification linear scaling
    base_verify = df['Verify Time (ms)'].iloc[0]
    y_verify = (base_verify/base_creds) * x_line
    plt.plot(x_line, y_verify, '--', color='red',
             label='Ideal Linear (Verify)', alpha=0.5)

    plt.xlabel('Number of Credentials')
    plt.ylabel('Execution Time (ms)')
    plt.title('PS Protocol Credential Scaling Analysis')
    plt.grid(True, alpha=0.3)
    plt.legend()

    # Save the visualization
    plt.savefig('ps_credential_scaling_analysis.png',
                dpi=300, bbox_inches='tight')
    plt.close()

    # Print detailed scaling analysis
    print("\nScaling Analysis:")
    print("-" * 100)
    print(f"{'Credentials':<12} | {'Prove Time':<20} | {'Verify Time':<20} | {'Prove Scaling':<15} | {'Verify Scaling':<15}")
    print("-" * 100)

    for i in range(1, len(df)):
        prev = df.iloc[i-1]
        curr = df.iloc[i]

        cred_ratio = curr['Credentials'] / prev['Credentials']
        prove_ratio = curr['Prove Time (ms)'] / prev['Prove Time (ms)']
        verify_ratio = curr['Verify Time (ms)'] / prev['Verify Time (ms)']

        prove_scaling = prove_ratio / cred_ratio
        verify_scaling = verify_ratio / cred_ratio

        print(f"{curr['Credentials']:<12} | {curr['Prove Time (ms)']:>8.2f} ms | {curr['Verify Time (ms)']:>8.2f} ms | {prove_scaling:>13.2f}x | {verify_scaling:>13.2f}x")


def main():
    print("Starting PS Protocol Benchmark Analysis")
    df = extract_benchmark_data()
    if df is not None:
        analyze_scaling(df)
        print("\nAnalysis complete! Check ps_credential_scaling_analysis.png for visualization")


if __name__ == "__main__":
    main()

Starting PS Protocol Benchmark Analysis
Analyzing benchmark data from: ../target/criterion/ps_equality_split

Found 5 proving and 5 verification configurations

Analyzing proving configuration: 30
Reading from: ../target/criterion/ps_equality_split/prove_credentials_30/new/estimates.json
✓ Successfully processed data for 30 credentials

Analyzing proving configuration: 2
Reading from: ../target/criterion/ps_equality_split/prove_credentials_2/new/estimates.json
✓ Successfully processed data for 2 credentials

Analyzing proving configuration: 5
Reading from: ../target/criterion/ps_equality_split/prove_credentials_5/new/estimates.json
✓ Successfully processed data for 5 credentials

Analyzing proving configuration: 20
Reading from: ../target/criterion/ps_equality_split/prove_credentials_20/new/estimates.json
✓ Successfully processed data for 20 credentials

Analyzing proving configuration: 10
Reading from: ../target/criterion/ps_equality_split/prove_credentials_10/new/estimates.json
✓ Suc

# PSUTT

In [3]:
import json
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


def extract_benchmark_data():
    """
    Extract benchmark data for both proving and verification times
    """
    results = []
    base_path = Path("../target/criterion/psutt_split")

    print(f"Analyzing benchmark data from: {base_path}")

    # Look for both prove and verify benchmarks
    prove_dirs = list(base_path.glob("prove_credentials_*"))
    verify_dirs = list(base_path.glob("verify_credentials_*"))

    print(
        f"\nFound {len(prove_dirs)} proving and {len(verify_dirs)} verification configurations")

    # Process proving benchmarks
    for bench_dir in prove_dirs:
        # Extract credential count from directory name
        creds = int(bench_dir.name.split('_')[2])

        estimates_file = bench_dir / "new" / "estimates.json"
        print(f"\nAnalyzing proving configuration: {creds}")
        print(f"Reading from: {estimates_file}")

        if estimates_file.exists():
            try:
                with open(estimates_file, 'r') as f:
                    data = json.load(f)

                # Extract timing metrics (converting nanoseconds to milliseconds)
                mean_time = data['mean']['point_estimate'] / 1_000_000
                std_dev = data['mean']['standard_error'] / 1_000_000

                # Find corresponding verification time
                verify_dir = next(d for d in verify_dirs if int(
                    d.name.split('_')[2]) == creds)
                verify_estimates = verify_dir / "new" / "estimates.json"

                with open(verify_estimates, 'r') as f:
                    verify_data = json.load(f)

                verify_mean = verify_data['mean']['point_estimate'] / 1_000_000
                verify_std = verify_data['mean']['standard_error'] / 1_000_000

                results.append({
                    'Credentials': creds,
                    'Prove Time (ms)': mean_time,
                    'Prove Std Dev': std_dev,
                    'Verify Time (ms)': verify_mean,
                    'Verify Std Dev': verify_std,
                    'Total Time (ms)': mean_time + verify_mean
                })
                print(f"✓ Successfully processed data for {creds} credentials")

            except Exception as e:
                print(f"× Error processing data: {e}")

    if results:
        # Create DataFrame and sort by credential count
        df = pd.DataFrame(results)
        df = df.sort_values('Credentials')

        # Save results
        df.to_csv('psutt_credential_scaling.csv', index=False)
        df.to_json('psutt_credential_scaling.json', orient='records', indent=2)

        print("\nScaling Analysis Summary:")
        print("-" * 70)
        print(df.to_string(index=False))

        return df
    else:
        print("\nNo data was collected!")
        return None


def analyze_scaling(df):
    """Analyze and visualize scaling behavior for both proving and verification"""
    plt.figure(figsize=(12, 8))

    # Plot proving time
    plt.scatter(df['Credentials'], df['Prove Time (ms)'],
                color='blue', label='Proving Time', s=100)
    if 'Prove Std Dev' in df.columns:
        plt.errorbar(df['Credentials'], df['Prove Time (ms)'],
                     yerr=df['Prove Std Dev'], fmt='none',
                     capsize=5, color='blue', alpha=0.5)

    # Plot verification time
    plt.scatter(df['Credentials'], df['Verify Time (ms)'],
                color='red', label='Verification Time', s=100)
    if 'Verify Std Dev' in df.columns:
        plt.errorbar(df['Credentials'], df['Verify Time (ms)'],
                     yerr=df['Verify Std Dev'], fmt='none',
                     capsize=5, color='red', alpha=0.5)

    # Plot total time
    plt.scatter(df['Credentials'], df['Total Time (ms)'],
                color='green', label='Total Time', s=100)

    # Compute and plot ideal linear scaling for both operations
    base_creds = df['Credentials'].iloc[0]
    x_line = np.linspace(0, df['Credentials'].max() * 1.1, 100)

    # Proving linear scaling
    base_prove = df['Prove Time (ms)'].iloc[0]
    y_prove = (base_prove/base_creds) * x_line
    plt.plot(x_line, y_prove, '--', color='blue',
             label='Ideal Linear (Prove)', alpha=0.5)

    # Verification linear scaling
    base_verify = df['Verify Time (ms)'].iloc[0]
    y_verify = (base_verify/base_creds) * x_line
    plt.plot(x_line, y_verify, '--', color='red',
             label='Ideal Linear (Verify)', alpha=0.5)

    plt.xlabel('Number of Credentials')
    plt.ylabel('Execution Time (ms)')
    plt.title('PSUtt Credential Scaling Analysis: Proving vs Verification')
    plt.grid(True, alpha=0.3)
    plt.legend()

    # Save the visualization
    plt.savefig('psutt_credential_scaling_analysis.png',
                dpi=300, bbox_inches='tight')
    plt.close()

    # Print scaling analysis
    print("\nScaling Analysis:")
    print("-" * 100)
    print(f"{'Credentials':<12} | {'Prove Time':<20} | {'Verify Time':<20} | {'Prove Scaling':<15} | {'Verify Scaling':<15}")
    print("-" * 100)

    for i in range(1, len(df)):
        prev = df.iloc[i-1]
        curr = df.iloc[i]

        cred_ratio = curr['Credentials'] / prev['Credentials']
        prove_ratio = curr['Prove Time (ms)'] / prev['Prove Time (ms)']
        verify_ratio = curr['Verify Time (ms)'] / prev['Verify Time (ms)']

        prove_scaling = prove_ratio / cred_ratio
        verify_scaling = verify_ratio / cred_ratio

        print(f"{curr['Credentials']:<12} | {curr['Prove Time (ms)']:>8.2f} ms | {curr['Verify Time (ms)']:>8.2f} ms | {prove_scaling:>13.2f}x | {verify_scaling:>13.2f}x")


def main():
    df = extract_benchmark_data()
    if df is not None:
        analyze_scaling(df)


if __name__ == "__main__":
    main()

Analyzing benchmark data from: ../target/criterion/psutt_split

Found 5 proving and 5 verification configurations

Analyzing proving configuration: 30
Reading from: ../target/criterion/psutt_split/prove_credentials_30/new/estimates.json
✓ Successfully processed data for 30 credentials

Analyzing proving configuration: 2
Reading from: ../target/criterion/psutt_split/prove_credentials_2/new/estimates.json
✓ Successfully processed data for 2 credentials

Analyzing proving configuration: 5
Reading from: ../target/criterion/psutt_split/prove_credentials_5/new/estimates.json
✓ Successfully processed data for 5 credentials

Analyzing proving configuration: 20
Reading from: ../target/criterion/psutt_split/prove_credentials_20/new/estimates.json
✓ Successfully processed data for 20 credentials

Analyzing proving configuration: 10
Reading from: ../target/criterion/psutt_split/prove_credentials_10/new/estimates.json
✓ Successfully processed data for 10 credentials

Scaling Analysis Summary:
-----